In [1]:
import pandas as pd
import numpy as np

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

#for easier display
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

In [2]:
# user reviews
df_ratings = pd.read_csv("user_reviews.csv", header=0)
print("Shape of df_ratings:", df_ratings.shape)

# the first column is user names, the rest are movie titles
user_col = df_ratings.columns[0]   # "User"
movie_cols = df_ratings.columns[1:]

# movie genres
df_genres = pd.read_csv("movie_genres.csv", header=0)
if "Unnamed: 0" in df_genres.columns:
    df_genres.drop(columns=["Unnamed: 0"], inplace=True)

#drop "Unnamed: 0" if it exists
if "Unnamed: 0" in df_ratings.columns:
    df_ratings.drop(columns=["Unnamed: 0"], inplace=True)

# now columns should be: [ "User", "Movie1", "Movie2", ... ]

user_col = df_ratings.columns[0]   # should now be "User"
movie_cols = df_ratings.columns[1:]  # all the rest


print("Shape of df_genres:", df_genres.shape)
print("\nSample of user_reviews.csv:")
display(df_ratings.head())
print("\nSample of movie_genres.csv:")
display(df_genres.head())

Shape of df_ratings: (600, 2002)
Shape of df_genres: (2000, 26)

Sample of user_reviews.csv:


User  The Net  Happily N'Ever After  Tomorrowland  American Hero  Das Boot  Final Destination 3  Licence to Kill  The Hundred-Foot Journey  The Matrix  Creature  The Basket  \
0  Vincent      0.0                   0.0           0.0            0.0       0.0                  0.0              0.0                       0.0         0.0       0.0         0.0   
1    Edgar      0.0                   0.0           0.0            0.0       0.0                  0.0              0.0                       0.0         0.0       0.0         0.0   
2  Addilyn      0.0                   0.0           0.0            0.0       0.0                  0.0              0.0                       0.0         0.0       0.0         0.0   
3   Marlee      0.0                   0.0           0.0            0.0       0.0                  0.0              0.0                       0.0         0.0       0.0         0.0   
4   Javier      0.0                   0.0           0.0            0.0       0.0                  0.0              0.0                       0.0         0.0       0.0         0.0   

   Star Trek: The Motion Picture  The Hunger Games  The Hurt Locker  Flatliners  The Blues Brothers  The Last Exorcism  Event Horizon  Vicky Cristina Barcelona  The Woman in Black  Galaxy Quest  \
0                            0.0               0.0              0.0         0.0                 0.0                0.0            0.0                       0.0                 0.0           0.0   
1                            0.0               0.0              0.0         0.0                 0.0                0.0            0.0                       0.0                 0.0           0.0   
2                            0.0               0.0              0.0         0.0                 0.0                0.0            0.0                       0.0                 0.0           0.0   
3                            0.0               0.0              0.0         0.0                 0.0                0.0            0.0                       0.0                 0.0           0.0   
4                            0.0               0.0              0.0         0.0                 0.0                0.0            0.0                       0.0                 0.0           0.0   

   Troy  The Messengers  Heaven Is for Real  The Iceman  Me and Orson Welles  The Blair Witch Project  Seven Samurai  Con Air  Little Children  Novocaine  Supercross  Pathology  Cutthroat Island  \
0   0.0             0.0                 0.0         0.0                  0.0                      0.0            0.0      0.0              0.0        0.0         0.0        0.0               0.0   
1   0.0             0.0                 0.0         0.0                  0.0                      0.0            0.0      0.0              0.0        0.0         0.0        0.0               0.0   
2   0.0             0.0                 0.0         0.0                  0.0                      0.0            0.0      0.0              0.0        0.0         0.0        0.0               0.0   
3   0.0             0.0                 0.0         0.0                  0.0                      0.0            0.0      0.0              0.0        0.0         0.0        0.0               0.0   
4   0.0             0.0                 0.0         0.0                  0.0                      0.0            0.0      0.0              0.0        0.0         0.0        0.0               0.0   

   Black Book  Nixon  Whale Rider  Bucky Larson: Born to Be a Star  Southpaw  The Oogieloves in the Big Balloon Adventure  Young Adult  Diary of a Wimpy Kid  Charly  The Wash  The Contender  \
0         0.0    0.0          0.0                              0.0       0.0                                          0.0          0.0                   0.0     0.0       0.0            0.0   
1         0.0    0.0          0.0                              0.0       0.0                                          0.0          0.0                   0.


Sample of movie_genres.csv:


,movie_title,genre_action,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,genre_documentary,genre_drama,genre_family,genre_fantasy,genre_film-noir,genre_history,genre_horror,genre_music,genre_musical,genre_mystery,genre_news,genre_reality-tv,genre_romance,genre_sci-fi,genre_short,genre_sport,genre_thriller,genre_war,genre_western
0,The Net,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,Happily N'Ever After,0,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tomorrowland,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,American Hero,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Das Boot,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [3]:
# we convert the user–movie table into a numeric matrix, where rows = users, columns = movies and missing ratings (originally 0) are replaced with each user's average rating

# create a separate matrix of just the ratings (excluding the user name)
rating_matrix = df_ratings[movie_cols].values.astype(float)
num_users, num_items = rating_matrix.shape

# replace zeros with each user’s average rating
# if a user has no ratings at all (all zeros), we fallback to the global mean
row_means = np.true_divide(
    rating_matrix.sum(axis=1),
    (rating_matrix != 0).sum(axis=1),
    where=(rating_matrix != 0).sum(axis=1) != 0
)
global_mean = np.nanmean(row_means)  # if some rows have no ratings
row_means = np.nan_to_num(row_means, nan=global_mean)  # fallback to global mean

#create a new matrix "R_filled" with zeros replaced
R_filled = rating_matrix.copy()
for i in range(num_users):
    # wherever R_filled[i,:] == 0, replace with row_means[i]
    mask = (R_filled[i, :] == 0)
    R_filled[i, mask] = row_means[i]

In [4]:
# we learn a low-rank approximation of R_filled. For example, 50 latent factors
# then reconstruct to get predicted ratings for every user–movie pair

n_components = 50
svd_model = TruncatedSVD(n_components=n_components, random_state=69)
U = svd_model.fit_transform(R_filled)  # shape = (num_users, n_components)
V = svd_model.components_              # shape = (n_components, num_items)

# reconstructed rating matrix
#R_approx[i, j] is the CF-predicted rating of user i for movie j
R_approx = np.dot(U, V)  # shape = (num_users, num_items)

In [5]:
# we expect df_genres to have something like movie_title, genre_action, genre_adventure, etc
# we compute a similarity matrix: sim[movie_i, movie_j]

if "movie_title" not in df_genres.columns:
    raise ValueError("The genres file must have a 'movie_title' column.")

genre_cols = [c for c in df_genres.columns if c.startswith("genre_")]
genre_matrix = df_genres[genre_cols].values  # shape = (num_movies, num_genres)

content_sim_matrix = cosine_similarity(genre_matrix, genre_matrix)

# build a dictionary from movie_title -> index in df_genres
title_to_index = dict(zip(df_genres["movie_title"], df_genres.index))

# build a dictionary from index in df_ratings columns -> movie_title
# because the columns of df_ratings (after the first "User") are also movie titles
col_index_to_title = dict(enumerate(movie_cols))

#for quick lookup from movie title -> column index in df_ratings
title_to_col_index = {title: idx for idx, title in col_index_to_title.items()}

In [6]:
#we use functions to easily modify parameters when testing so we dont need to change in multiple places
def recommend_hybrid_svd(user_name, alpha=0.7, top_n=5):
    #locate the user index
    user_df_row = df_ratings[df_ratings[user_col] == user_name]
    if user_df_row.empty:
        print(f"User '{user_name}' not found.")
        return []
    user_index = user_df_row.index[0]
    
    # the user's original ratings
    user_ratings = rating_matrix[user_index, :]
    
    # identify the user’s "high-rated" movies (which is >=4) from the original data
    high_rated_movie_indices = np.where(user_ratings >= 4.0)[0]
    
    # build a set of all rated items
    rated_movie_indices = np.where(user_ratings != 0)[0]
    
    # for each unrated movie, get CF score and content-sim score
    results = []
    
    for m_idx in range(num_items):
        if user_ratings[m_idx] == 0:  # meaning user hasn't rated it
            # collaborative Filtering part
            cf_score = R_approx[user_index, m_idx]
            
            # content-based part: average sim to the user's high-rated movies
            # we use df_genres index -> content_sim_matrix
            # we need to find the row in df_genres that corresponds to this movie
            movie_title = col_index_to_title[m_idx]
            if movie_title in title_to_index:
                cb_idx = title_to_index[movie_title]
            else:
                # if the movie isn't in df_genres, fallback to 0 similarity
                cb_idx = None

            if cb_idx is not None and len(high_rated_movie_indices) > 0:
                sim_scores = []
                for hm_idx in high_rated_movie_indices:
                    hm_title = col_index_to_title[hm_idx]
                    if hm_title in title_to_index:
                        hm_cb_idx = title_to_index[hm_title]
                        sim_val = content_sim_matrix[hm_cb_idx, cb_idx]
                        sim_scores.append(sim_val)
                if len(sim_scores) > 0:
                    avg_sim = np.mean(sim_scores)
                else:
                    avg_sim = 0.0
            else:
                avg_sim = 0.0
            
            # hybrid combination with our set alpha
            hybrid_val = alpha * cf_score + (1 - alpha) * avg_sim
            
            results.append((m_idx, cf_score, avg_sim, hybrid_val))
    
    # sort by the hybrid score
    results.sort(key=lambda x: x[3], reverse=True)
    
    # return top_n
    top_recs = results[:top_n]
    
    # convert indices back to movie titles
    final_recs = []
    for (idx, cfs, sim, hyb) in top_recs:
        final_recs.append((col_index_to_title[idx], cfs, sim, hyb))
    return final_recs

In [7]:
users_to_recommend = ["Vincent", "Edgar", "Addilyn", "Marlee", "Javier"]

for u in users_to_recommend:
    print(f"\nRecommendations for {u}:")
    top_items = recommend_hybrid_svd(u, alpha=0.7, top_n=5)
    for rank, (title, cf_val, sim_val, hyb_val) in enumerate(top_items, 1):
        print(f"{rank}. {title} | CF={cf_val:.2f} | Sim={sim_val:.2f} | Hybrid={hyb_val:.2f}")


Recommendations for Vincent:
1. Cheap Thrills | CF=4.14 | Sim=0.41 | Hybrid=3.02
2. 21 Grams | CF=4.05 | Sim=0.57 | Hybrid=3.01
3. The Fourth Kind | CF=4.19 | Sim=0.19 | Hybrid=2.99
4. 500 Days of Summer | CF=4.05 | Sim=0.49 | Hybrid=2.98
5. Vicky Cristina Barcelona | CF=4.03 | Sim=0.53 | Hybrid=2.98

Recommendations for Edgar:
1. Hoffa | CF=4.15 | Sim=0.35 | Hybrid=3.01
2. Made | CF=4.12 | Sim=0.41 | Hybrid=3.01
3. Tycoon | CF=4.10 | Sim=0.44 | Hybrid=3.00
4. You Can Count on Me | CF=4.03 | Sim=0.55 | Hybrid=2.99
5. Primary Colors | CF=4.04 | Sim=0.46 | Hybrid=2.97

Recommendations for Addilyn:
1. Morning Glory | CF=3.95 | Sim=0.58 | Hybrid=2.94
2. The Stewardesses | CF=3.93 | Sim=0.60 | Hybrid=2.93
3. Teenage Mutant Ninja Turtles: Out of the Shadows | CF=3.99 | Sim=0.39 | Hybrid=2.91
4. The Fisher King | CF=3.90 | Sim=0.53 | Hybrid=2.89
5. Robot & Frank | CF=3.91 | Sim=0.50 | Hybrid=2.89

Recommendations for Marlee:
1. The Good Thief | CF=3.63 | Sim=0.49 | Hybrid=2.69
2. Striptease 